# Using LLMs in Python

In [1]:
import os

import dotenv
import PIL.Image

import google.generativeai as genai

In [2]:
loaded: bool = dotenv.load_dotenv()
print("Loaded environment variables." if loaded else "Could not load environment variables.")

Loaded environment variables.


In [3]:
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [4]:
model_type = 'models/gemini-1.5-flash'
model = genai.GenerativeModel(model_type)
model_info = genai.get_model(model_type)
print(f"Model name: {model_info.name}\nModel version: {model_info.version}\nInput token limit: {model_info.input_token_limit}\nOutput token limit: {model_info.output_token_limit}")

Model name: models/gemini-1.5-flash
Model version: 001
Input token limit: 1000000
Output token limit: 8192


In [39]:
# Counting tokens
prompt = "I want to design a building facade that helps with carbon-capture and fits in with the aesthetic of central London."
model.count_tokens(prompt)

total_tokens: 23

In [40]:
# Prompting
response = model.generate_content(prompt)
print(response.text)

## Carbon-Capturing Facade for Central London: Blending Aesthetics and Sustainability

Here's a concept for a facade that blends carbon capture with the architectural heritage of central London:

**Concept:**

* **Inspired by Victorian Era Architecture:** The facade takes inspiration from London's iconic Victorian architecture, specifically the use of ornate details, terracotta, and brickwork. 
* **Integrated Carbon Capture System:**  The facade incorporates a system of bio-filters and photocatalytic materials that actively capture carbon dioxide from the air. 
* **Aesthetic Integration:** The carbon capture elements are seamlessly integrated into the design, disguised as decorative features.
* **Adaptive Design:**  The system is designed to be modular and adaptable, allowing for customization to different building sizes and shapes.

**Specific Elements:**

* **Terracotta Panels:**  Ornamental terracotta panels with integrated photocatalytic coatings. These panels act as a passive carb

### Using LangChain

[Langchain](https://python.langchain.com/docs/introduction/) is a framework for developing applications powered by large language models (LLMs). In this example, [ChatGoogleGenerativeAI](https://python.langchain.com/docs/integrations/chat/google_generative_ai/) is used to exemplify interaction with the same model.

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

**Note:** It is recommended to experiment with model hyperparameters, such as [temperature](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/adjust-parameter-values#temperature), to find the right configuration for your application. Temperature influences the degree of randomness or 'creativity' in the model's output. Lower values will lead to more deterministic responses, while higher values will result in more 'creative' responses.

In [8]:
model_temperature = 0.2
llm = ChatGoogleGenerativeAI(model=model_type, google_api_key=GEMINI_API_KEY, temperature=model_temperature)

In [41]:
response = llm.invoke(prompt)

In [42]:
print(response.content)

## Carbon-Capturing Facade for Central London: A Blend of Functionality and Aesthetics

Here's a concept for a building facade that combines carbon capture technology with the classic elegance of central London:

**Concept:**

* **Inspired by Victorian Architecture:** The facade draws inspiration from the intricate details and ornate designs of Victorian buildings in London, using traditional materials like brick, stone, and cast iron. This ensures a seamless integration with the surrounding architecture.
* **Integrated Bio-Wall System:** The facade incorporates a vertical bio-wall system, featuring a variety of native plants and mosses known for their carbon absorption capabilities. This living wall acts as a natural air filter, capturing CO2 and releasing oxygen.
* **Smart Glass Panels:**  The facade features strategically placed smart glass panels that can adjust their transparency based on sunlight levels and building energy needs. This optimizes natural light penetration while min

### RAG with LangChain

![RAG with LangChain](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

<i>Figure Source: LangChain Tutorials - Building a Retrieval Augmented Generation (RAG) App; URL: https://python.langchain.com/docs/tutorials/rag/#retrieval-and-generation</i>

In [23]:
import pandas as pd

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [45]:
# Load and split a PDF file
document_path = "../resources/document_examples/The_Evolution_and_Change_of_Building_Facades_Research_D_Turgut_Cikis.pdf"
document_loader = PyPDFLoader(document_path)
document_pages = document_loader.load_and_split()
print(f"Loaded the specified document: {len(document_pages)} pages")

Loaded the specified document: 121 pages


In [47]:
# Visualzie the contents of the 5th page of the document
print(document_pages[20].page_content)

10 hung on the building structure, usually from floor to floor. It can provide a variety of 
exterior appearances but characterized by narrowly spaced vertical and horizontal caps 
with glass or metal infill panels. These systems provide a finished exterior appearance 
and most often a semi-finished interior as well (Quirouette 1982). They are also 
designed to accommodate structural deflections, control wind-driven rain and leakage, 
minimize the effects of solar radiation and provide for maintenance-free long term 
performance. Most of today's metal curtain wall systems are constructed of lightweight 
aluminum, although some may be of steel.  
 
2.2. Historical Background 
 
Even walls present in building traditions since the beginning of human 
existence, the intention of creating large openings on it was first seen in the gothic 
architecture.  But the concept of fully glazed wall developed fitfully soon after this 
period. Therefore it was an architectural idea only awaiting neces

In [48]:
# Create chunks from the loaded PDF pages
chunk_size=8000
chunk_overlap=1000

character_splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=1000)
text_chunks = character_splitter.split_text("\n\n".join(str(page.page_content) for page in document_pages))
print(f"Created {len(text_chunks)} chunks of size {chunk_size} from the original {len(document_pages)} document pages.")

Created 32 chunks of size 8000 from the original 121 document pages.


In [50]:
# Set the model that will embed the text chunks
embeddings_model = "models/embedding-001"
embeddings = GoogleGenerativeAIEmbeddings(model=embeddings_model, google_api_key=GEMINI_API_KEY)

# Create a vectorstore from the text chunks, using the set model to embed them
number_of_chunks_to_retrieve_per_query = 3
vectorstore_retriever = Chroma.from_texts(texts=text_chunks, embedding=embeddings).as_retriever(search_kwargs={"k":number_of_chunks_to_retrieve_per_query})

In [51]:
# Create a chain for prompting, retrieving chunks from the vectorstore and using them in the llm to produce an answer
llm_qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore_retriever, return_source_documents=True)

In [71]:
prompt = {
    "query" : "I want to design a facade for a large residential building in a hot climate in Sourthern Europe. The aim is to maximize the energy efficiency of the building through the facade design while minimizing the weight of the facade. What type of cladding system should be used? What percentage of it should be glazed?"
}

In [72]:
response = llm_qa_chain.invoke(prompt)

In [73]:
print(response["result"])

Given the requirements for a large residential building in a hot Southern European climate, aiming for energy efficiency and minimal weight, here's a breakdown of the facade design considerations:

**Cladding System:**

* **Ventilated Facade:** This system is ideal for hot climates. It creates an air gap between the cladding and the building's insulation, allowing for natural ventilation and reducing heat gain. 
    * **Materials:**
        * **Lightweight Metal Panels:** Aluminum or galvanized steel panels are excellent choices due to their durability, low weight, and ability to be easily formed into various shapes.
        * **High-Performance Ceramic Tiles:**  These offer excellent thermal insulation and can be designed to reflect solar radiation.
        * **Composite Panels:** Combining materials like wood, metal, and insulation can create a lightweight and energy-efficient system.

**Glazing Percentage:**

* **Minimize Glazing:**  In a hot climate, maximizing solar heat gain thro

In [83]:
# The specific data from the document that was used to generate the prompt can be retrieved from the response
print("\n".join([document.page_content for document in response["source_documents"]]))

51 appearance. Zinc panels should be well in ventilated on the on the non-exposed 
surfaces. Water from condensation or leaking can dissolve the metal relatively quickly.  
C. Titanium is also a popular but costly material, which is rarely used in 
distinguished example of architecture. But because of the cost factor it doesn’t used 
commercially in industrialized cladding system. 
 
2.5.2. Transparent Curtain Walls 
 
The very first material evocation of curtain wall is glass. As it is known glass is 
one of the most common and also one of the oldest building materials. Its feature and 
material properties are developed during the course of history. Today it is possible to 
find different types of glass that is produced for specific needs in building market. But 
according to Wiggenton, The development in glass enormous but they are concentrated 
on finishes and coatings, and gases for insulated units, and small manipulations on float 
glass. Spurred by energy concerns, aesthetics and

### Returning structured output

In order to ensure that the model response is compatible with the continuation of the application pipeline, [a specific output structure can be imposed](https://python.langchain.com/docs/how_to/structured_output/) - such as Pydantic classes, JSON schemas or custom definitions.

In [84]:
from typing import Optional
from pydantic import BaseModel, Field

In [85]:
# Create a Pydantic class to enforce the structure of the model's response
class FacadeSpecifications(BaseModel):
    """The facade specifications recommended by the LLM in accordance with the provided documentation."""
    
    cladding_system: str = Field(description="The cladding system of the building.")
    glazing_system: str = Field(description="The glazing system of the building.")
    predominant_material: str = Field(description="The predominant cladding material in the facade.")
    glazing_percentage: int = Field(description="The percentage of the facade that should be comprised of glazing.")


In [ ]:
# USE THIS RESOURCE TO CREATE STRUCTURED OUTPUTS: https://stackoverflow.com/questions/76822673/langchain-querying-a-document-and-getting-structured-output-using-pydantic-with